## Character Generation with RNN

Use RNN to predict the next character based on the script it learn from. Using a character predictive model that will take as input a variable length sequence and predict the next character.

In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

### Select example for training
Upload a file to train the model.

In [2]:
from google.colab import files

path_to_file = list(files.upload().keys())[0]

Saving myplay.txt to myplay.txt


### Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding
The text isn't encoded yet, need to encode each unique character as a different integer.



In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# seehow part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


make a function can convert numeric values to text


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### Create Training Examples
need to split the text into many shorter sequences so that we can pass them to the model as training examples. 

The training examples will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Thefirst step will be to create a stream of characters from training text.

In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next use the batch method to turn this stream of characters into batches of desired length.

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Then use these sequences of length 101 and split them into input and output.

In [10]:
def split_input_target(chunk):  # for example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # use map to apply the above function to every entry

In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. 
# Instead, it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building Model
Use an embedding layer a LSTM and one dense layer that contains a node for each unique character in training data. The dense layer will give a probability distribution over all nodes.

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Loss Function
Create loss function for the problem. The model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

Before that, have a look at a sample input and the output from untrained model. This is to understand what the model is giving.



In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 4.58100345e-04 -1.02854683e-03 -5.83313173e-04 ... -3.47035890e-03
    4.04777937e-03  2.86678108e-03]
  [ 2.70669442e-03 -5.62708313e-03 -3.25422548e-03 ... -1.58827170e-03
    6.70511555e-03  4.17502923e-03]
  [ 4.78090486e-04 -1.14017650e-02  7.34310597e-04 ... -3.99519503e-03
    5.11288643e-03  3.93899390e-03]
  ...
  [-3.97634087e-03  2.30691349e-03  8.52939580e-03 ... -2.91690696e-03
   -4.73884633e-04  3.09741264e-03]
  [-2.45897751e-03  1.89101405e-03  5.48218284e-03 ... -6.95572933e-03
    1.82843523e-03  6.65706117e-03]
  [-2.56600813e-03 -2.96165957e-03  7.47612584e-03 ... -4.09485400e-03
    9.11735743e-03  4.34376486e-03]]

 [[ 2.55026133e-03 -4.60104551e-03 -2.96687777e-03 ...  8.22856731e-04
    3.54148704e-03  1.51580526e-03]
  [ 5.91808930e-06 -3.29818809e-03 -1.91698794e-03 ... -3.01954034e-03
    4.86237230e-04 -2.80385860e-03]
  [-4.36783675e-03  2.29032361e-03 -3.68375797e-03 ... -1.13501796e-04
    3.44175100e-03  3.66489450e-03]
  ...
  [-4.597

In [16]:
# examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.0004581  -0.00102855 -0.00058331 ... -0.00347036  0.00404778
   0.00286678]
 [ 0.00270669 -0.00562708 -0.00325423 ... -0.00158827  0.00670512
   0.00417503]
 [ 0.00047809 -0.01140177  0.00073431 ... -0.0039952   0.00511289
   0.00393899]
 ...
 [-0.00397634  0.00230691  0.0085294  ... -0.00291691 -0.00047388
   0.00309741]
 [-0.00245898  0.00189101  0.00548218 ... -0.00695573  0.00182844
   0.00665706]
 [-0.00256601 -0.00296166  0.00747613 ... -0.00409485  0.00911736
   0.00434376]], shape=(100, 65), dtype=float32)


In [17]:
# look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

# its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 4.5810034e-04 -1.0285468e-03 -5.8331317e-04  1.6671882e-03
  1.7399129e-03  2.3488322e-04  3.7117098e-03 -4.0513147e-03
 -2.2102529e-03 -3.5234692e-03  9.2147500e-04  1.2171179e-02
 -1.4905143e-03  1.5539157e-03  2.8563039e-03  6.1290162e-03
  3.7021055e-03 -7.0861308e-05  2.1743812e-03 -3.7422623e-03
  8.0642523e-04  7.5293076e-04  1.6361514e-04  2.5302861e-03
  5.7541109e-03  2.9855031e-03  1.5998038e-04 -4.6333540e-03
  2.5480273e-03 -5.1612146e-03  2.1964749e-03  7.9888115e-03
  1.0311938e-03  5.6060781e-03 -4.6766857e-03  2.9508350e-03
 -7.6783923e-03  1.1686387e-04 -2.7739250e-03  3.7489850e-03
 -9.1465918e-04 -3.7596163e-03 -2.5719744e-03  1.4722748e-03
 -1.5196645e-03  9.5738110e-04  1.3109144e-03 -1.7247294e-03
 -2.6796106e-03 -1.7208604e-03 -7.8862747e-03  2.6688848e-03
  3.2249801e-03 -3.0408942e-04  2.3463145e-03 -2.7613190e-04
  1.2559878e-03 -5.0163823e-03 -6.7465389e-03  6.9095497e-03
  1.5874860e-03 -5.1879790e-03 -3.4703589e-03  4.0477794e-03
  2.866781

In [18]:
# to determine the predicted character, need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

":LhlPmU\nttF?hY,OnySI3$.,!z-$m ATrXIvQFidnuX\nyjuqC;geGQLoNG.x'Gkun:X3DA,zzeulYVyGBIJXpD;,UOvkPxV$u& J"

now to create a loss function that can compare that output to the expected output and give numeric value representing how close the two were. 

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Compile Model

In [20]:
model.compile(optimizer='adam', loss=loss)

### Creating Checkpoints
Setup and configure model to save checkpoinst. This will allow to load model from a checkpoint and continue training it.

In [21]:
# Directory to save the checkpoints
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

### Training Model

In [22]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 63ms/step - loss: 2.5319
Epoch 2/50
172/172 [==============================] - 12s 65ms/step - loss: 1.8491
Epoch 3/50
172/172 [==============================] - 12s 65ms/step - loss: 1.6111
Epoch 4/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4833
Epoch 5/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4066
Epoch 6/50
172/172 [==============================] - 13s 70ms/step - loss: 1.3518
Epoch 7/50
172/172 [==============================] - 13s 71ms/step - loss: 1.3071
Epoch 8/50
172/172 [==============================] - 14s 74ms/step - loss: 1.2665
Epoch 9/50
172/172 [==============================] - 13s 71ms/step - loss: 1.2304
Epoch 10/50
172/172 [==============================] - 13s 70ms/step - loss: 1.1923
Epoch 11/50
172/172 [==============================] - 14s 72ms/step - loss: 1.1562
Epoch 12/50
172/172 [==============================] - 14s 72ms/step - loss: 1.1171
E

### Load the Model
Rebuild the model from a checkpoint using a batch_size of 1 to feed one peice of text to the model and have it make a prediction.

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once model is trained, find the **lastest checkpoint** that stores the models weights using the following line.

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

load **any checkpoint** by specifying the exact file to load.

In [25]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

### Generate Text
Use function provided by tensorflow to generate some text using any starting string we'd like.

In [26]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # pass the predicted character as the next input to model
      # along with previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Let's see how this work
Let's see how this work about me.

GREMIO:
As will I get them: thou hast made good
have worn themselves; but which you swear
Than my poor brother away to-night
Have sometime consent of such distression.

RIVERS:
By heaven, my heart am such a place-banishment.

FRIAR LAURENCE:
O, shalt thou so? Away, I am more than me?
I am sorry that she not been dread; or Dorter?

Embost:
Gainst your daughter
Goo's torments, the Duke of Norfolk,
And that this spoils of hence plucks him in a queen:
I'll not part up up again. While she lives
My hand to see her: so we will discover all
The game of Mantua:--baried steel'd with treble dee.

ISABELLA:
Let me hear you speak for thee.

PERDITA:
Even he, for Tarwick with him in the versal.

DUKE VINCENTIO:
It is not yet.

LUCENTIO:
I may, and will, it were worse prisoner.

EDWARD:
Nay, 
